# Analyzing Student Behavior in an SQL Course

## Introduction

This project explores student behavior in an online SQL course, structured into modules, lessons, and practical steps. Each step requires students to submit correct solutions, and many steps allow multiple attempts. The project focuses on querying and analyzing this educational data using SQL within a Python (Pandas + SQLite) environment.

The analysis is performed in a Jupyter Notebook. The goal is to demonstrate both technical skills in SQL and data interpretation abilities.

## Project Objectives

• Analyze student progress and performance across modules and lessons.

• Identify difficult steps where students require more attempts.

• Understand overall course completion trends.

• Practice SQL skills in a real-world dataset context.

## Dataset Scheme

![Описание](images/scheme.png)

## Import Libraries & Load Data

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
# Load CSV files from Google Drive

module = pd.read_csv('https://drive.google.com/uc?id=1YUcuTYEky5VCHeeGzPCbfPpkLP_ibLf8')
lesson = pd.read_csv('https://drive.google.com/uc?id=1XWx5-itALqUwvZzHmXqAR9TjHsEvPXDc')
step = pd.read_csv('https://drive.google.com/uc?id=1nmAWf8jLq7PpsOXvXpTsYISyTw057BMv')
student = pd.read_csv('https://drive.google.com/uc?id=1uDV0CnW6qsi_7SJI58NacLePJFFW8-23')
step_student = pd.read_csv('https://drive.google.com/uc?id=1jkicD7P4gdgjmVrTHvgziv98sFp--Fwy')

In [ ]:
# Create in-memory SQLite database

conn = sqlite3.connect(':memory:')

In [ ]:
# Load tables into the database

module.to_sql('module', conn, index=False)
lesson.to_sql('lesson', conn, index=False)
step.to_sql('step', conn, index=False)
student.to_sql('student', conn, index=False)
step_student.to_sql('step_student', conn, index=False)

## Task 1: Calculate Student Progress and Award Certificates

__Objective:__
Determine each student's progress in the course as a percentage of correctly completed steps out of the total number of distinct steps attempted across all students. Based on progress, assign one of the following:

"Distinction Certificate" – if progress = 100%

"Certificate" – if progress ≥ 80%

Sort results by descending progress and then alphabetically by student name.

__Comment: Why we use a Python variable instead of an SQL one?__

In standard SQL (especially in MySQL or PostgreSQL), we could use a session variable like __@total_steps__ directly in the query.
However, SQLite (used here in-memory via Jupyter) does not support user-defined session variables.

Therefore, we first calculate the total number of steps using a separate SQL query and store the result in a Python variable, which we then interpolate into the next SQL query using an f-string. 

In [ ]:
# Step 1: Get the total number of distinct steps attempted in the course
total_steps = pd.read_sql("""
    SELECT COUNT(DISTINCT step_id) AS total FROM step_student
""", conn).iloc[0, 0]

print(f"Total number of distinct steps: {total_steps}")

In [ ]:
# Step 2: Use the total number of steps to calculate each student's progress and assign certificates

query_task_1 = f"""
SELECT 
    student_name,
    progress,
    CASE 
        WHEN progress = 100 THEN 'Distinction certificate'
        WHEN progress >= 80 THEN 'Certificate'
        ELSE 'No certificate'
    END AS certificate
FROM (
    SELECT 
        student_name,
        ROUND(COUNT(DISTINCT step_id) * 100.0 / {total_steps}) AS progress
    FROM student 
    JOIN step_student USING(student_id)
    WHERE result = 'correct'
    GROUP BY student_name
) query_in
ORDER BY progress DESC, student_name;
"""

progress_df = pd.read_sql(query_task_1, conn)
progress_df.head()
# to display all rows uncomment this command:
# progress_df

### Conclusion:

In the first task, we identified the certification status of each student who completed the course. Each student was categorized as either passed with distinction, passed with a regular certificate, or did not receive a certificate.

This classification allows us to further analyze the distribution of students across these groups. 

## Task 2: Calculate Average Lesson Completion Time

__Objective:__
Estimate how long, on average, it takes students to complete each lesson in hours.

__Requirements:__

• Ignore attempts that took more than 4 hours, assuming the student was inactive.

• Assign a sequential integer to each row. For each lesson, specify the module number and its position within the module.

In [ ]:
query_task_2 = """

-- Calculate the total time a student spent on each step

WITH time_per_step AS (
    SELECT 
        step_id, 
        student_id, 
        SUM(submission_time - attempt_time) AS step_time
    FROM step_student
    WHERE (submission_time - attempt_time) <= 4 * 3600
    GROUP BY step_id, student_id
),

-- Aggregate total time spent by each student on each lesson

time_per_lesson AS (
    SELECT 
        student_id, 
        lesson_id, 
        SUM(step_time) AS lesson_time
    FROM lesson
    JOIN step USING (lesson_id)
    JOIN time_per_step USING (step_id)
    GROUP BY student_id, lesson_id
)

-- Calculate the average time spent in hours per lesson across all students
SELECT 
    ROW_NUMBER() OVER (ORDER BY AVG(lesson_time)) AS row_num,
    module_id || '.' || lesson_position || ' ' || lesson_name AS lesson,
    ROUND(AVG(lesson_time) / 3600, 2) AS avg_time
FROM time_per_lesson
JOIN lesson USING (lesson_id)
GROUP BY lesson_id;
"""

avg_lesson_time = pd.read_sql(query_task_2, conn)
avg_lesson_time

### Conclusion

In the third task, we calculated the average time students spent completing each lesson.

The result suggests that Lesson 2.2 is the most time-efficient for students, while Lesson 2.4 requires more effort on average. These insights can help instructors identify which lessons are more challenging or time-consuming and adjust the content or support accordingly.

## Task 3: Detailed Attempt Analysis for Student_59

__Objective:__
Provide a detailed breakdown of all attempts made by student_59.

__Output Columns:__

• Student name

• Step – Concatenated position: ModuleNumber.LessonPosition.StepPosition

• Attempt number (ordered by submission time)

• Result

• Attempt_Duration

• Relative_Time

__Additional Rules:__

If the attempt took more than 1 hour, replace its duration with the student's average attempt time across all steps (excluding attempts >1 hour).

Sort first by step_id, then by submission time.

In [ ]:
query_task_3 = """

-- Calculate the average time between attempt and submission for student_59, 
-- exluding attempts where this time is more than 1 hour

WITH mean_time AS (
    SELECT 
        student_name, 
        ROUND(AVG(submission_time - attempt_time)) AS mean
    FROM student
    JOIN step_student USING (student_id)
    WHERE student_name = 'student_59' AND (submission_time - attempt_time) <= 3600
),

-- Select all attempts by student_59, calculate the number of each attempt per step,
-- and replace long attempt durations (> 1 hour) with the student's average time

select_student AS (
    SELECT 
        student_name,
        step_id,
        module_id || '.' || lesson_position || '.' || step_position AS stp_num,
        ROW_NUMBER() OVER (PARTITION BY step_id ORDER BY submission_time) AS attempt_num,
        result,
        CASE 
            WHEN (submission_time - attempt_time) <= 3600 THEN (submission_time - attempt_time)
            ELSE mean
        END AS att_time
    FROM student 
    JOIN mean_time USING (student_name)
    JOIN step_student USING (student_id)
    JOIN step USING (step_id)
    JOIN lesson USING (lesson_id)
    WHERE student_name = 'student_59'
    ORDER BY step_id, attempt_num
)

-- For each attempt, display the attempt time and relative time spent on the attempt

SELECT 
    student_name, 
    stp_num, 
    attempt_num, 
    result,
    TIME(att_time, 'unixepoch') AS attempt_time,
    ROUND(att_time * 100.0 / SUM(att_time) OVER (PARTITION BY  step_id), 2) AS relative_time
FROM select_student
ORDER BY step_id, 3
"""

st_59_detailed = pd.read_sql(query_task_3, conn)
st_59_detailed.head()

# to display all rows uncomment this command:
# st_59_detailed

### Conclusion:

In Task 4, we examined the behavior of a specific student while solving course steps. For each attempt, we calculated the time taken and how that time compares to the first (longest) attempt on the same step — using a metric called relative time.

Key observations:

• The first attempt is usually the longest, often taking up to 100% of the total step time.

• In most cases, subsequent attempts are significantly faster, often taking less than 5% of the original time.

This pattern suggests that after initial exploration and understanding, the student becomes more confident and efficient in solving the step.

These findings highlight how student behavior evolves through trial and error. Analyzing such patterns can help instructors detect moments of struggle or disengagement and improve the structure of complex steps.

## Task 4: Student Ranking per Module

__Objective:__
For each module, calculate a relative performance rating for each student based on the number of steps they successfully completed.

Calculate relative rating as: Relative rating = student steps / max steps in module * 100

Sort by: module number, rating descending, student name.

In [ ]:
query_task_4 = """

-- Count the number of correctly completed steps by each student in each module

WITH student_step_count AS (
    SELECT 
        module_id, 
        student_name, 
        student_id,
        COUNT(DISTINCT step_id) AS correct_steps_count
    FROM step_student 
        JOIN step USING (step_id)
        JOIN lesson USING (lesson_id)
        JOIN student USING (student_id)
    WHERE result = 'correct'
    GROUP BY module_id, student_name
)

-- Calculate the relative rating within each module as a percentage of the maximum steps completed

SELECT 
    module_id, 
    student_name, 
    correct_steps_count AS rate,
    ROUND(correct_steps_count * 100.0 / MAX(correct_steps_count) OVER (PARTITION BY module_id), 1) 
        AS relative_rate
FROM student_step_count
ORDER BY module_id, rate DESC, student_id;
"""

relative_rating = pd.read_sql(query_task_4, conn)
relative_rating.head()

# to display all rows uncomment this command:
# relative_rating

### Conclusion:

In Module 1, student performance is exceptionally consistent: nearly all students (over 50) achieved a 100% completion rate, meaning they successfully completed all 11 steps of the module. A few students had slightly lower completion, mostly in the 90–80% range, indicating they missed only one or two steps.

In contrast, Module 2 shows much greater variation in student engagement and completion. While some students again completed all steps (100% completion), many others dropped off earlier. Completion rates in Module 2 span a wide spectrum—from full completion down to just 2 out of 21 steps (9.5%). This disparity suggests that Module 2 might have been more challenging, longer, or less engaging for some students.

Overall, this analysis reveals a drop in full engagement from Module 1 to Module 2, potentially highlighting areas where improvements in content design or pacing may be beneficial in maintaining student motivation and progress.

## Task 5: Identify Atypical Step Completion Patterns

__Objective:__
Detect students with non-standard learning behavior based on how they interact with steps.

Define 3 Behavior Groups:

Group I – Students who submit a correct attempt, followed by an incorrect one

Group II – Students who make multiple correct attempts on the same step

Group III – Students who make only incorrect attempts on a step

Sort by: group number (I–III), then descending steps_count, then by student name.

__Goal:__
Help identify which assignments might confuse or intrigue students and may benefit from revision or clarification.

In [ ]:
query_task_5 = """

-- Steps where the student first submitted a correct answer and then a wrong one

WITH group_I AS (
    SELECT 
        student_name,
        step_id,
        CASE 
            WHEN LEAD(result) OVER (
                PARTITION BY student_id, step_id 
                ORDER BY submission_time
            ) = 'wrong' AND result = 'correct' 
            THEN 1
            ELSE 0
        END AS error_after_success
    FROM step_student 
    JOIN student USING (student_id)
),

-- Steps where the student submitted multiple correct answers

group_II AS (
    SELECT 
        student_name,
        step_id
    FROM step_student
    JOIN student USING (student_id)
    WHERE result = 'correct'
    GROUP BY student_name, step_id
    HAVING COUNT(*) >= 2
),

-- Steps where the student never submitted a correct answer

group_III AS (
    SELECT 
        student_name,
        step_id
    FROM step_student 
    JOIN student s USING (student_id)
    GROUP BY student_name, step_id
    HAVING SUM(CASE WHEN result = 'correct' THEN 1 ELSE 0 END) = 0
)

-- Combine all three groups and count steps per student in each group

SELECT 
    'I' AS group_name, 
    student_name, 
    COUNT(step_id) AS steps_count
FROM group_I
WHERE error_after_success = 1
GROUP BY student_name

UNION

SELECT 
    'II', 
    student_name, 
    COUNT(step_id)
FROM group_II
GROUP BY student_name

UNION

SELECT 
    'III', 
    student_name, 
    COUNT(step_id)
FROM group_III
GROUP BY student_name

ORDER BY group_name, steps_count DESC, student_name;

"""

atypical_groups = pd.read_sql(query_task_5, conn)
atypical_groups.head()

# to display all rows uncomment this command:
# atypical_groups

### Conclusion:

Based on the classification of student interactions with course steps, we observe three key behavioral patterns:

__Group I — Correct Answer Followed by a Wrong One__
Interpretation: This behavior may indicate that students were experimenting or trying to understand how the testing system works. For example, they might intentionally try different inputs after solving the task correctly to explore system boundaries or test hypotheses.

__Group II — Multiple Correct Submissions for the Same Step__
Interpretation: This suggests that students were refining their solutions. They might be optimizing their code, trying different approaches, or aiming for better efficiency and clarity—even after receiving a correct result. This reflects an active learning mindset.

__Group III — Never Submitted a Correct Answer__
Interpretation: These students likely struggled with the specific content of the step or lost motivation. These steps may require further review to ensure clarity, support materials, or guidance for learners who get stuck.